In [ ]:
using BSON, Dates, DelimitedFiles, Downloads, Flux, Printf, Plots #import libraries
include("neural.jl");

In [ ]:
function generate_inout(ρ_profiles, c1_profiles, ϕ_profiles; window_width, dx)
    window_bins = 2 * round(Int, window_width / dx) + 1
    ρ_windows_all = Vector{Vector{Float32}}()
    c1_values_all = Vector{Float32}()
    ϕ_functions_all = Vector{Vector{Float32}}()
    for (ρ, c1, ϕ) in zip(ρ_profiles, c1_profiles, ϕ_profiles)
        ρ_windows = generate_windows(ρ; window_bins)
        ϕ_func = generate_phi(ϕ,ρ)
        s = 40 # use every s'th training example if you want to speed up the process
        for i in collect(1:s:length(c1)) 
            if !isfinite(c1[i])
                continue
            end
            push!(ρ_windows_all, ρ_windows[:,i])
            push!(c1_values_all, c1[i])
            push!(ϕ_functions_all, ϕ_func[:,i])
            
        end
    end
    reduce(hcat, ρ_windows_all), c1_values_all', reduce(hcat, ϕ_functions_all)
end

The following cell generates the training data from simulation data in the directory `Simulations`, where each file contains the (thermally scaled) local chemical potential $\beta \mu_\mathrm{loc}(x) = \beta (\mu - V_\mathrm{ext}(x))$, the density profile $\rho(x)$ and the pair potential $\beta \phi(r)$.
The function `read_sim_data` utilizes this data to calculate the one-body direct correlation function $c_1(x) = \ln(\rho(x))- \beta \mu_\mathrm{loc}(x)$.
The training sets containing the density-window and the pair potential as input and the $c_1$-value as output are created with the help of `generate_inout`.       

In [ ]:
#Downloads.download("https://myfiles.uni-bayreuth.de/filr/public-link/file-download/043490a392e1752f01931b17e3711848/93318/1417842602830201006/Simulations.tar", "Simulations.tar")
#run(`tar xf Simulations.tar`)
datadir = "Simulations" #this contains training data
ρ_profiles, c1_profiles, ϕ_profiles = read_sim_data(datadir);
ρ_windows, c1_values, ϕ_functions = generate_inout(ρ_profiles, c1_profiles, ϕ_profiles; window_width=2.0, dx=0.01)
size(ρ_windows), size(c1_values), size(ϕ_functions) #the last item in the tuple is the number of training samples we use here

In [ ]:
#This is an example contained in 'Simulations'
sim = rand(readdir(datadir, join = true))
xs, βµloc, ρ, βϕ = eachcol(readdlm(sim))
display(plot(xs, βµloc, ylabel = "βµloc", xlabel = "x/σ"))
display(plot(xs, ρ, ylabel = "ρ", xlabel = "x/σ"))
display(plot(xs[1:150], βϕ[1:150], ylabel = "βϕ", xlabel = "r/σ"))

The cell below shows the training procedure of a neural metadensity functional with the previously prepared data.

In [ ]:
using CUDA
ρ_windows, c1_values, ϕ_functions = (ρ_windows, c1_values, ϕ_functions) |> gpu

model = Chain(
    Dense(size(vcat(ρ_windows, ϕ_functions))[1] => 512, softplus),
    Dense(512 => 256, softplus),
    Dense(256 => 128, softplus),
    Dense(128 => 64, softplus),
    Dense(64 => 32, softplus),
    Dense(32 => 1) 
) |> gpu

display(model) #architecture  

opt = Flux.setup(Adam(), model) #optimizer

loader = Flux.DataLoader((vcat(ρ_windows, ϕ_functions), c1_values), batchsize=256, shuffle=true)|> gpu  

loss(m, x, y) = Flux.mse(m(x), y) #mean squared error 
metric(m, x, y) = Flux.mae(m(x), y) #mean absolute error
get_learning_rate(epoch; initial=0.0001, rate=0.03, wait=5) = epoch < wait ? initial : initial * (1 - rate)^(epoch - wait)

model_savefile = "own_model.bson"
println("Saving model to $(model_savefile)")
for epoch in 1:150 
    learning_rate = get_learning_rate(epoch)
    Flux.adjust!(opt, learning_rate)      
    @printf "Epoch: %3i (learning_rate: %.2e)..." epoch learning_rate; flush(stdout)
    Flux.train!(loss, model, loader, opt) 
    @printf " loss: %.5f, metric: %.5f\n" loss(model, vcat(ρ_windows, ϕ_functions), c1_values) metric(model, vcat(ρ_windows, ϕ_functions), c1_values); flush(stdout)
    model = model |> cpu
    BSON.@save model_savefile model
    model = model |> gpu
end

model